In [376]:
## Calculates how likely are teams to make playoffs for LEC Summer 2020
## Does not simulate tiebreaker

In [307]:
import requests
from bs4 import BeautifulSoup

In [308]:
data = requests.get("https://lol.gamepedia.com/LEC/2020_Season/Summer_Season")
f = open("lec_site", "w+")
f.write(data.text)
f.close()

In [310]:
soup = BeautifulSoup(data.text, 'html.parser')

In [311]:
rows = soup.find_all("tr", class_="mdv-allweeks")

In [312]:
# def getname(team1, team2):
#    name = ""
#    if(team1 < team2):
#        name = team1 + team2
#    else:
#        name = team2 + team1
#    return name

In [363]:
teams = ['G2', 'VIT', 'SK', 'RGE', 'FNC', 'MAD', 'OG', 'MSF', 'XL', 'S04']
second_wins = {}
wins = {}
head2head = {}
total = 0
score = {}
std = []
for i in range(10):
    std.append([])
for team in teams:
    second_wins[team] = 0
    wins[team] = 0
    score[team] = 0
for team1 in teams:
    for team2 in teams:
        if team1 == team2:
            continue
        head2head[(team1, team2)] = 0
        head2head[(team2, team1)] = 0

upcoming = []

In [364]:
match = 0

for row in rows:
    try:
        teams = row.find_all("td")
        # print(teams[0].text, "vs", teams[1].text, teams[2].text.split(" "))
        team1 = teams[0].text[2: ]
        team2 = teams[1].text[2: ]
        winner = ""
        teamName = getname(team1, team2)
        if(teams[2].text.split(" ")[0] == '1'):
            winner = team1
            head2head[(team1, team2)] += 1
            head2head[(team2, team1)] -= 1
        elif (teams[2].text.split(" ")[0] == '0'):
            winner = team2
            head2head[(team2, team1)] += 1
            head2head[(team1, team2)] -= 1
        else:
            upcoming.append([team1, team2])
        wins[winner] += 1
        match += 1
        if match > 45:
            second_wins[winner] = second_wins[winner] + 1   
    except:
        continue
        

In [365]:
print(wins, '\n', second_wins, '\n', head2head)

{'G2': 10, 'VIT': 7, 'SK': 9, 'RGE': 12, 'FNC': 8, 'MAD': 12, 'OG': 6, 'MSF': 6, 'XL': 8, 'S04': 7} 
 {'G2': 5, 'VIT': 3, 'SK': 5, 'RGE': 5, 'FNC': 4, 'MAD': 5, 'OG': 2, 'MSF': 1, 'XL': 4, 'S04': 6} 
 {('G2', 'VIT'): 0, ('VIT', 'G2'): 0, ('G2', 'SK'): 0, ('SK', 'G2'): 0, ('G2', 'RGE'): 2, ('RGE', 'G2'): -2, ('G2', 'FNC'): 2, ('FNC', 'G2'): -2, ('G2', 'MAD'): 0, ('MAD', 'G2'): 0, ('G2', 'OG'): -2, ('OG', 'G2'): 2, ('G2', 'MSF'): 2, ('MSF', 'G2'): -2, ('G2', 'XL'): -1, ('XL', 'G2'): 1, ('G2', 'S04'): 0, ('S04', 'G2'): 0, ('VIT', 'SK'): 0, ('SK', 'VIT'): 0, ('VIT', 'RGE'): 0, ('RGE', 'VIT'): 0, ('VIT', 'FNC'): -2, ('FNC', 'VIT'): 2, ('VIT', 'MAD'): 0, ('MAD', 'VIT'): 0, ('VIT', 'OG'): 2, ('OG', 'VIT'): -2, ('VIT', 'MSF'): -1, ('MSF', 'VIT'): 1, ('VIT', 'XL'): -2, ('XL', 'VIT'): 2, ('VIT', 'S04'): 0, ('S04', 'VIT'): 0, ('SK', 'RGE'): 0, ('RGE', 'SK'): 0, ('SK', 'FNC'): -1, ('FNC', 'SK'): 1, ('SK', 'MAD'): 0, ('MAD', 'SK'): 0, ('SK', 'OG'): 2, ('OG', 'SK'): -2, ('SK', 'MSF'): 0, ('MSF', 'SK

In [366]:
upcoming

[['VIT', 'MSF'], ['MAD', 'S04'], ['RGE', 'OG'], ['G2', 'XL'], ['FNC', 'SK']]

In [367]:
draw = False

In [368]:
def tie_breaker(teams, second_wins, head2head):
    # print(teams)
    global draw
    if(len(teams) == 2):
        team1 = teams[0]
        team2 = teams[1]
        if(head2head[(team1, team2)] == 0):
            # head2head is same
            if(second_wins[team1] == second_wins[team2]):
                # play tiebreaker
                print(" 2 way tie ", team1, team2)
                std[0].append(team2)
                std[1].append(team1)
                return [team1 , team2]
            else:
                if second_wins[team1] > second_wins[team2]:
                    return [team1, team2]
                return [team2, team1]
        else:
            if head2head[(team1, team2)] > 0:
                return [team1 , team2]
            return [team2, team1]
    else:
        if(len(teams) >= 3):
            for team in teams:
                other = []
                for other_team in teams:
                    if team != other_team:
                        other.append(other_team)
                superior = True
                for other_team in other:
                    if head2head[(team, other_team)]< 0:
                        superior = False
                if superior:
                    st = [team] + tie_breaker(other, second_wins, head2head)
                    return st
        print(len(teams) ," way tie ", teams)
        return teams
                

In [369]:
def fix_tie(wins, second_wins, head2head):
    #print(wins)
    global score, draw
    d = {}
    for team in wins:
        no_wins = wins[team]
        if no_wins not in d:
            d[no_wins] = [team]
        else:
            d[no_wins].append(team)
    #print(d)
    final_standings = []
    for wins in d:
        teams = d[wins]
        if len(teams) == 1:
            final_standings.append(teams[0])
        else:
            order = tie_breaker(teams, second_wins, head2head)
            final_standings += order
    print(final_standings)
    for i in range(6):
        score[final_standings[i]] += 1
    for i in range(10):
        std[i].append(final_standings[i])
        
    
        

In [370]:
def play(gameNo, wins, second_wins, head2head):
    global total
    if(gameNo > len(upcoming)- 1):
        total += 1
        final_wins = {k: v for k, v in sorted(wins.items(), key=lambda item: -item[1])}
        fix_tie(final_wins, second_wins, head2head)
        return
    win1 = wins.copy();
    win2 = wins.copy();
    second_wins1 = second_wins.copy();
    second_wins2 = second_wins.copy();
    head2head1 = head2head.copy();
    head2head2 = head2head.copy();
    team1 = upcoming[gameNo][0];
    team2 = upcoming[gameNo][1];
    teamName = getname(team1, team2);
    # team1 wins 
    win1[team1] += 1
    second_wins1[team1] += 1
    head2head1[(team1, team2)] += 1
    head2head1[(team2, team1)] -= 1
    play(gameNo + 1, win1, second_wins1, head2head1)
    
    # team2 wins
    win2[team2] += 1
    second_wins2[team2] += 1
    head2head2[(team1, team2)] -= 1
    head2head2[(team2, team1)] += 1
    play(gameNo + 1, win2, second_wins2, head2head2)
    
    
    

In [371]:
play(0, wins, second_wins, head2head)

 2 way tie  RGE MAD
['RGE', 'MAD', 'G2', 'FNC', 'SK', 'XL', 'VIT', 'S04', 'OG', 'MSF']
 2 way tie  RGE MAD
['RGE', 'MAD', 'G2', 'SK', 'FNC', 'XL', 'VIT', 'S04', 'OG', 'MSF']
 2 way tie  RGE MAD
['RGE', 'MAD', 'G2', 'FNC', 'XL', 'SK', 'VIT', 'S04', 'OG', 'MSF']
 2 way tie  RGE MAD
['RGE', 'MAD', 'SK', 'G2', 'XL', 'FNC', 'VIT', 'S04', 'OG', 'MSF']
['MAD', 'RGE', 'G2', 'FNC', 'SK', 'XL', 'VIT', 'S04', 'OG', 'MSF']
['MAD', 'RGE', 'G2', 'SK', 'FNC', 'XL', 'VIT', 'S04', 'OG', 'MSF']
['MAD', 'RGE', 'G2', 'FNC', 'XL', 'SK', 'VIT', 'S04', 'OG', 'MSF']
['MAD', 'RGE', 'SK', 'G2', 'XL', 'FNC', 'VIT', 'S04', 'OG', 'MSF']
['RGE', 'MAD', 'G2', 'FNC', 'SK', 'XL', 'S04', 'VIT', 'OG', 'MSF']
['RGE', 'MAD', 'G2', 'SK', 'S04', 'FNC', 'XL', 'VIT', 'OG', 'MSF']
['RGE', 'MAD', 'G2', 'FNC', 'XL', 'SK', 'S04', 'VIT', 'OG', 'MSF']
['RGE', 'MAD', 'SK', 'G2', 'XL', 'S04', 'FNC', 'VIT', 'OG', 'MSF']
 2 way tie  RGE MAD
['RGE', 'MAD', 'G2', 'FNC', 'SK', 'XL', 'S04', 'VIT', 'OG', 'MSF']
 2 way tie  RGE MAD
['RGE', '

In [372]:
total

32

In [373]:
score2  = {k: v for k, v in sorted(score.items(), key=lambda item: -item[1])}
for team in score2:
    print(team, score2[team], score2[team]/total)

G2 32 1.0
SK 32 1.0
RGE 32 1.0
MAD 32 1.0
FNC 28 0.875
XL 26 0.8125
S04 10 0.3125
VIT 0 0.0
OG 0 0.0
MSF 0 0.0


In [375]:
teams = ['G2', 'VIT', 'SK', 'RGE', 'FNC', 'MAD', 'OG', 'MSF', 'XL', 'S04']
for i in range(10):
    print(str(i + 1) + " :")
    for team in teams:
        print(team, (1.0 * std[i].count(team))/total," ",  end='')
    print()

1 :
G2 0.0  VIT 0.0  SK 0.0  RGE 0.75  FNC 0.0  MAD 0.75  OG 0.0  MSF 0.0  XL 0.0  S04 0.0  
2 :
G2 0.0  VIT 0.0  SK 0.0  RGE 0.75  FNC 0.0  MAD 0.75  OG 0.0  MSF 0.0  XL 0.0  S04 0.0  
3 :
G2 0.75  VIT 0.0  SK 0.25  RGE 0.0  FNC 0.0  MAD 0.0  OG 0.0  MSF 0.0  XL 0.0  S04 0.0  
4 :
G2 0.25  VIT 0.0  SK 0.25  RGE 0.0  FNC 0.5  MAD 0.0  OG 0.0  MSF 0.0  XL 0.0  S04 0.0  
5 :
G2 0.0  VIT 0.0  SK 0.25  RGE 0.0  FNC 0.125  MAD 0.0  OG 0.0  MSF 0.0  XL 0.5  S04 0.125  
6 :
G2 0.0  VIT 0.0  SK 0.25  RGE 0.0  FNC 0.25  MAD 0.0  OG 0.0  MSF 0.0  XL 0.3125  S04 0.1875  
7 :
G2 0.0  VIT 0.25  SK 0.0  RGE 0.0  FNC 0.125  MAD 0.0  OG 0.0  MSF 0.25  XL 0.1875  S04 0.1875  
8 :
G2 0.0  VIT 0.375  SK 0.0  RGE 0.0  FNC 0.0  MAD 0.0  OG 0.0  MSF 0.25  XL 0.0  S04 0.375  
9 :
G2 0.0  VIT 0.375  SK 0.0  RGE 0.0  FNC 0.0  MAD 0.0  OG 0.5  MSF 0.0  XL 0.0  S04 0.125  
10 :
G2 0.0  VIT 0.0  SK 0.0  RGE 0.0  FNC 0.0  MAD 0.0  OG 0.5  MSF 0.5  XL 0.0  S04 0.0  
